In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, Dropout, Input, BatchNormalization, concatenate
from keras.models import Model, Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import cv2
import tensorflow as tf
import imgaug as ia
import imgaug.augmenters as iaa
from albumentations import (
    RandomRotate90,Flip
)
import scipy.io
import difflib
from scipy import spatial

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
print('This should list a GPU: ',tf.test.gpu_device_name())
os.chdir('/home/xx652230/Downloads/')

In [ ]:
def getBand_Power(freqs, power, lower, upper):
    ''' Sum band power within desired frequency range '''
    low_idx = np.array(np.where(freqs <= lower)).flatten()
    up_idx = np.array(np.where(freqs > upper)).flatten()
    band_power = np.sum(power[low_idx[-1]:up_idx[0]])/len(power[low_idx[-1]:up_idx[0]])
    #return [band_power,freqs[low_idx[-1]:up_idx[0]]]
    return band_power
def getFiveBands_Power(freqs, power):
    ''' Calculate 5 bands power '''
    theta_power = getBand_Power(freqs, power, 3, 7)
    slow_alpha_power = getBand_Power(freqs, power, 8, 10)
    alpha_power = getBand_Power(freqs, power, 10, 13)
    beta_power = getBand_Power(freqs, power, 14, 29)
    gamma_power = getBand_Power(freqs, power, 30, 47)

    return theta_power, slow_alpha_power, alpha_power, beta_power, gamma_power

def category(y):
        y1=[0.0]*5
        if y[0] > 7 and y[1] > 7:
                y1[0] = 1.0
        elif y[0] > 7 and y[1] < 3:
                y1[1] = 1.0
        elif y[0] < 3 and y[1] > 7:
                y1[3] = 1.0
        elif y[0] < 3 and y[1] < 3:
                y1[2] = 1.0
        else:
            y1[4] = 1.0
        return y1
    
def normalise(x):
    x=(x-np.mean(x))/np.std(x)
    return x
def normalize(mat):
    return mat/255

In [ ]:
num_features=64
modele = Sequential([
    Conv2D(filters=64, kernel_size=5, activation='relu', input_shape=(120, 120, 1)),
    Dropout(0.2),
    BatchNormalization(),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=32, kernel_size=3, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=16, kernel_size=5, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(5, activation='softmax')
])
modele.load_weights("/home/xx652230/Downloads/model/eeg/deap.hdf5")
modele.compile(optimizer='adam', loss='categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
num_features = 64
width, height = 48, 48
modelf = Sequential([
        Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1)),  # 1
        Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1)),  # 1
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 2
        Dropout(0.5),

        Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 3
        BatchNormalization(),
        Conv2D(2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 4
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 5
        Dropout(0.5),

        Conv2D(2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 3
        BatchNormalization(),
        Conv2D(2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 4
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 5
        Dropout(0.5),

        Conv2D(2 * 2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 6
        BatchNormalization(),
        Conv2D(2 * 2 * 2 * num_features, kernel_size=(3, 3), activation='relu', padding='same'),  # 7
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),  # 8
        Dropout(0.5),

        Flatten(),

        Dense(2 * 2 * 2 * num_features, activation='relu'),  # 9
        Dropout(0.4),
        Dense(2 * 2 * num_features, activation='relu'),  # 10
        Dropout(0.4),
        Dense(2 * num_features, activation='relu'),  # 10
        Dropout(0.5),

        Dense(7, activation='softmax')

])
modelf.load_weights("/home/xx652230//Downloads/model/face/final_model_face.hdf5") 
modelf.compile(optimizer='adam', loss='categorical_crossentropy' , metrics=['accuracy'])


In [ ]:
ref=[]
for i in range(5):
    x=pd.read_csv("./features1"+str(i)+".csv")
    ref.append(x["values"])

In [ ]:
m1=Model(inputs=modele.inputs,outputs=modele.layers[-2].output)
m1.compile(optimizer="adam", loss='categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
amigos_classes=["neutral","disgust","happiness","other","angry","fear","sadness"]
model_classes=["neutral","happiness","sadness","angry","disgust","fear","other"]
eeg_classes=["q1","q2","q3","q4","neu"]
    
def featureCompare(F1):
    global sm_scores,simi
    stat=0
    stats=0
    stat1=0
    for i in range(5):
        similarity = 1 - spatial.distance.cosine(F1,ref[i])
        sm=difflib.SequenceMatcher(None,F1,ref[i])
        sm_scores.append(sm.ratio()*100)
        simi.append(similarity*100)
        if similarity*100>50 and i!=4:
            if stat<similarity*100:
                stat=similarity*100
                stat1=i+1
        elif  similarity*100>50 and i==4 :
            if stat<similarity*100:
                stat=similarity*100
                stat1=i+1
    return stat1        
            
def multi(scores, score1,score2):
    global order2,score
    one=np.where(scores==max(scores))[0]
    index=eeg_class(one,2)
    two=np.where(score1==max(score1))[0]  
    th=np.where(score2==max(score2))[0]
    generate=[0.0]*4
    if len(two)!=0:
        if two[0]==one[0]: 
            order2=order2+1
            generate[one[0]]=1.0
            return 1,generate
    elif one[0] in th and np.all((np.array(th)==0))==False:
            order2=order2+1
            generate[one[0]]=1.0
            return 1,generate
    else:
        return 0,[0,0,0,0]

def final(eeg, face, amigos,feature_vec):
    global order, order1, order2, order3, compare_index
    q1 = eeg[0] * (face[1])
    q2 = eeg[1] * (face[3] + face[5] + face[4])
    q3 = eeg[2] * face[2]
    q4 = eeg[3]  
    q = [q1, q2, q3, q4]
    order3 = order3 + 1  # total frames
    amigos2 = np.where(amigos[5:] == 1.0)
    max_eeg = np.where(eeg == max(eeg))
    _,individual=eeg_class(max_eeg[0][0],2)
    max_face = np.where(face == max(face))
    quad_face = face_class(max_face[0], 2)
    amigos1, quad_ind = eeg_class(amigos[0:5], 1)
    score=[0.0]*4
    # total frames matched
    keys = []
    keys2 = []
    frame_limit = 0
    for i in amigos2[0]:
        k1, k2 = face_class(i, 1)
        keys.append(k1)
        keys2.append(k2)
        if amigos_classes[i] == model_classes[int(max_face[0])] and amigos_classes[i] != "neutral" and frame_limit == 0:
            order1 = order1 + 1
            frame_limit = 1

        elif k2 == quad_face or k2=="any" and quad_face!="neu" and frame_limit == 0:
                if  k2=="any":
                    order1 = order1 + 1
                    frame_limit = 1
                else:
                    order1 = order1 + 1
                    frame_limit = 1
    
    # total eegs
    eeg_limit = 0
    quad_eeg_list = []
    for i in max_eeg[0]:
        quad_eeg,_ = eeg_class(i, 2)
        quad_eeg_list.append(quad_eeg)
        if amigos1 == quad_eeg and amigos1 != "neu" and eeg_limit == 0:
            order = order + 1
            eeg_limit = 1
            #compare_index=given_index
        elif quad_eeg in keys2 and quad_eeg!="neu" and eeg_limit == 0 or "any" in keys2:
                order = order + 1
                eeg_limit = 1
                #compare_index=given_index
                
        #else:
         #   announce=featureCompare(feature_vec)
         #   if announce!=5:
          #      order=order+1
           #     eeg_limit=1
           
    modal_limit = 0
    amigos_score_quadrant=[0.0]*4
    if quad_ind!=4:
            amigos_score_quadrant[quad_ind]=1.0
    for i in amigos2[0]:
        amigos_score_emotion_quadrant = [0.0]*4  
        if i == 2:
            amigos_score_emotion_quadrant[0] = 1.0
        elif i == 1 or i == 4 or i == 5 :
            amigos_score_emotion_quadrant[1] = 1.0
        elif i == 6:
            amigos_score_emotion_quadrant[2] = 1.0
        elif i== 3:
            amigos_score_emotion_quadrant=[1.0 for i in range(4)]
        x1,x2 = multi(q, amigos_score_quadrant,np.array(amigos_score_emotion_quadrant))
        if x1==1:
            if max(x2)>max(score):
                score=x2
                modal_limit=1
            
               
    return eeg_limit,frame_limit,score, individual,modal_limit

def eeg_class(input, lock):
    if lock == 2:
        y = 0
        yy=[0.0]*5
        if input == 0:
            y = "q1"
            yy[0]=1.0
        elif input == 1:
            y = "q2"
            yy[1]=1.0
        elif input == 2:
            y = "q3"
            yy[2]=1.0
        elif input == 3:
            y = "q4"
            yy[3]=1.0
        elif input == 4:
            y = "neu"
            yy[4]=1.0
    
        return y,yy
    else:
        y1 = []
        y2 = 0
        if input[0] > 7 and input[1] > 7:
            y1 = "q1"
            y2 = 0
        elif input[0] > 7 and input[1] < 3:
            y1 = "q2"
            y2 = 1
        elif input[0] < 3 and input[1] > 7:
            y1 = "q4"
            y2 = 3
        elif input[0] < 3 and input[1] < 3:
            y1 = "q3"
            y2 = 2
        else:
            y1 = "neu"
            y2 = 4
        return y1, y2

def face_class(i, j):
    if j == 1:
        key1 = 0
        key2 = 0
        if i == 0:
            key1 = "neutral"
            key2 = "neu"
        elif i == 1:
            key1 = "disgust"
            key2 = "q2"
        elif i == 6:
            key1 = "sadness"
            key2 = "q3"
        elif i == 4:
            key1 = "angry"
            key2 = "q2"
        elif i == 2:
            key1 = "happiness"
            key2 = "q1"
        elif i == 5:
            key1 = "fear"
            key2 = "q2"
        else:
            key1 = "other"
            key2 = "any"
        return key1, key2
    else:
        key1 = 0
        key2 = 0
        if i == 0:
            key1 = "neutral"
            key2 = "neu"
        elif i == 4:
            key1 = "disgust"
            key2 = "q2"
        elif i == 2:
            key1 = "sadness"
            key2 = "q3"
        elif i == 3:
            key1 = "angry"
            key2 = "q2"
        elif i == 1:
            key1 = "happiness"
            key2 = "q1"
        elif i == 5:
            key1 = "fear"
            key2 = "q2"
        else:
            key1 = "other"
            key2 = "any"
        return key1, key2


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import KFold
from keras.preprocessing.sequence import TimeseriesGenerator
from scipy import spatial


In [ ]:
import csv
def write_csv(filename,path,ids):
    for img in filename:
        value = img.flatten()
        #with open("./redo/"+str(path)+str(ids)+".csv", 'a') as f:
        with open("./redo/"+str(path)+"20.csv", 'a') as f:
            writer = csv.writer(f)
            writer.writerow(value)

# Extracting training set with sampling process

In [ ]:
order=0
order1 = 0
order2 = 0
order3=0
x_face=[]
x_eeg=[]
y_quad=[]
y_all=[]
y_score=[]
count=0
eeg_frames=[]
sm_scores=[]
simi=[]
rejected=0
for q in os.listdir("./amigos/eeg/"):
  if not q.startswith("."):
        print(q)
        mat = scipy.io.loadmat('./amigos/eeg/'+str(q))
        _, _, q1 = str(q).split("_")
        q2, _ = str(q1).split(".")
        _,q3=str(q2).split("P")
    
        for i in os.listdir("./amigos/frames/eeg/"+str(q2)):
            if not i.startswith(".") :
                d = mat["VideoIDs"][0]
                e=int(np.where(d == i)[0])
                eeg_result=[]
                amigos = mat["labels_selfassessment"][0][e][0]
                outcome = category(amigos[0:5])
                eegs=[]
                compare_index=0
                
                for k1 in range(len(os.listdir("./amigos/frames/eeg/"+str(q2)+"/"+str(i)))):
                    #total=np.zeros((1,1))
                    #for l in os.listdir("./amigos/eeg_frames/"+str(q2)+"/"+str(i)+"/"+str(k)):
                    image=cv2.imread("./amigos/frames/eeg/"+str(q2)+"/"+str(i)+"/frames"+str(k1)+".jpg")
                    resized=cv2.resize(image,(120,120))
                    #total=cv2.addWeighted(total,0.5,resized,0.5,0)
                    gray1 = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
                    gray1=normalize(np.array(gray1))
                    #order1+=1
                    xt = np.expand_dims(gray1, axis=-1)
                    res = modele.predict(np.array([xt,]))[0]
                    res1=m1.predict(np.array([xt,]))[0]
                    eeg_result.append(res)
                    eeg_frames.append(res1)
                    eegs.append(xt)
                    
                temp=0
                temp1=0
                check1=[]
                check2=[]
                check3=[]
                check4=[]
                check5=[]
                for k2 in range(len(os.listdir("./amigos/frames/2sec/"+str(q2)+"/" + str(i)))):
                    if k2<2*len(eeg_result):#k!=0 and k%2==0:
                        image = cv2.imread("./amigos/frames/2sec/"+str(q2)+"/" + str(i) + "/frames" + str(k2) + ".jpg")
                        faceCascade = cv2.CascadeClassifier(
                            cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
                        faces = faceCascade.detectMultiScale(
                            image,
                            scaleFactor=1.3,
                            minNeighbors=3,
                            minSize=(30, 30)
                        )

                        face_result = []
                        temp1 = temp1 + 1
                        if len(faces) == 1 :
                            for (x, y, w, h) in faces:
                                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                                face_crop = image[y:y + h, x:x + w]
                                output1 = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
                                imgg1 = cv2.resize(output1, (48, 48))
                                imgg1=normalize(np.array(imgg1))
                                output2 = np.expand_dims(imgg1, axis=-1)
                                output22 = np.reshape(output2, (1, 48, 48, 1))
                                z = modelf.predict(output22)
                                R1,R2,R3,R4,R5 = final(eeg_result[int(k2/2)], z[0], amigos,eeg_frames[int(k2/2)])
                                if R1==1 or R2==1 or R5==1:
                                    if temp<40: 
                                        temp=temp+1
                                    #R=[i for i in R2]
                                        check1.append(eegs[int(k2/2)])
                                        check2.append(output2)
                                        check3.append(outcome)
                                        check4.append([tp for tp in R3])
                                        check5.append([tx for tx in R4])
                                    
                print(str(i))
                if len(check1)==40:
                    print("yes")
                    print("clip:",str(i))
                    eegg=np.reshape(np.array(check1),(len(check1),120,120))
                    facee=np.reshape(np.array(check2),(len(check1),48,48))
                    output=np.reshape(np.array(check3),(len(check1),5))
                    output1=np.reshape(np.array(check4),(len(check1),4))
                    output2=np.reshape(np.array(check5),(len(check1),5))
                    write_csv(eegg,"eeg",str(q2))
                    write_csv(facee,"face",str(q2))
                    write_csv(output,"quad",str(q2))
                    write_csv(output1,"score",str(q2))
                    write_csv(output2,"model",str(q2))
            )
           
                        
                else:
                    rejected+=1
                        
                                    
                                 
        count+=1                
                       
        print("eegs,frames,total",order,order1,order3)
   
print(rejected)    

# Extracting test set

In [ ]:
order=0
order1 = 0
order2 = 0
order3=0
x_face=[]
x_eeg=[]
y_quad=[]
y_all=[]
y_score=[]
count=0
eeg_frames=[]
sm_scores=[]
simi=[]
rejected=0
for q in os.listdir("./amigos/eeg20/"):
  lst=["Data_Preprocessed_P12.mat"]  
  if not q.startswith(".") and q not in lst:
        print(q)
        mat = scipy.io.loadmat('./amigos/eeg20/'+str(q))
        _, _, q1 = str(q).split("_")
        q2, _ = str(q1).split(".")
        _,q3=str(q2).split("P")
        for i in os.listdir("./amigos/frames/eeg/"+str(q2)):
            if not i.startswith(".") :
                d = mat["VideoIDs"][0]
                e=int(np.where(d == i)[0])
                eeg_result=[]
                amigos = mat["labels_selfassessment"][0][e][0]
                outcome = category(amigos[0:5])
                ind=[ig for ig in range(len(os.listdir("./amigos/frames/eeg/"+str(q2)+"/" + str(i))))]
                ind2=[]
                eegs=[]
                temp=0
                temp1=0
                check1=[]
                check2=[]
                check3=[]
                check4=[]
                check5=[] 
                for k1 in ind:
                    
                    image=cv2.imread("./amigos/frames/eeg/"+str(q2)+"/"+str(i)+"/frames"+str(k1)+".jpg")
                    resized=cv2.resize(image,(120,120))
                    #total=cv2.addWeighted(total,0.5,resized,0.5,0)
                    gray1 = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
                    gray1=normalize(np.array(gray1))
                    #order1+=1
                    xt = np.expand_dims(gray1, axis=-1)
                    eegs.append(xt)
                    
                np.random.shuffle(ind)
                for k2 in ind:
                        image = cv2.imread("./amigos/frames/2sec/"+str(q2)+"/" + str(i) + "/frames" + str(k2) + ".jpg")
                        faceCascade = cv2.CascadeClassifier(
                            cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
                        faces = faceCascade.detectMultiScale(
                            image,
                            scaleFactor=1.3,
                            minNeighbors=3,
                            minSize=(30, 30)
                        )

                        face_result = []
                        temp1 = temp1 + 1
                        if len(faces) == 1 :
                            for (x, y, w, h) in faces:
                                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                                face_crop = image[y:y + h, x:x + w]
                                output1 = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
                                imgg1 = cv2.resize(output1, (48, 48))
                                imgg1=normalize(np.array(imgg1))
                                output2 = np.expand_dims(imgg1, axis=-1)
                                output22 = np.reshape(output2, (1, 48, 48, 1))
                                #z = modelf.predict(output22)
                                #R1,R2,R3,R4,R5 = final(eeg_result[int(k2/2)], z[0], amigos,eeg_frames[int(k2/2)])
                                #if R1==1 or R2==1 or R5==1:
                                if temp<40: 
                                        temp=temp+1
                                        #ind2.append(k2)
                                    #R=[i for i in R2]
                                        check1.append(eegs[int(k2/2)])
                                        check2.append(output2)
                                        check3.append(outcome)
                
                   
                
                                    
                print(str(i))
                if len(check1)==40:
                    print("yes")
                    print("clip:",str(i))
                    eegg=np.reshape(np.array(check1),(len(check1),120,120))
                    facee=np.reshape(np.array(check2),(len(check1),48,48))
                    output=np.reshape(np.array(check3),(len(check1),5))
                    #output1=np.reshape(np.array(check4),(len(check1),4))
                    #output2=np.reshape(np.array(check5),(len(check1),5))
                    write_csv(eegg,"eeg",str(q2))
                    write_csv(facee,"face",str(q2))
                    write_csv(output,"quad",str(q2))
                    #write_csv(output1,"score",str(q2))
                    #write_csv(output2,"model",str(q2))